# Develop a method for harmonizing data from OzWALD

Data is here: https://dapds00.nci.org.au/thredds/catalog/ub8/au/catalog.html

Requirements:
* Must be reproducible in an operational context i.e. minumum of fuss to rerun the whole process each year, but first off we need a ~20yr archive to build the models and run historic predictions
* For now, run at 5 km resolution
* Intermediate files are fine, but lets keep the number of steps to a minimum
* Some variables are already computed by OzWALD, but others need to be either computed on-the-fly or saved and stored as intermediate files.
* Many of the pre-computed variables available in OzWALD require resampling spatially and temporally
* A python environment is required, but should be a small as possible (but will undoubtedly still be cumbersome)
* There is a soft requirement that the model be built on features as close to possible as the published 'AusEFlux' article in Biogeosciences.



In [ ]:
import os
import pandas as pd
import xarray as xr
import numpy as np
from odc.geo.xr import assign_crs
from odc.geo.geobox import zoom_out

import warnings
warnings.simplefilter(action='ignore')

import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import start_local_dask, round_coords

In [ ]:
client = start_local_dask(mem_safety_margin='2Gb')
client

## Analysis Parameters

In [ ]:
base = '/g/data/ub8/au/'
results='/g/data/os22/chad_tmp/AusEFlux/data/interim/'
years = [str(i) for i in range(2003,2023)]

### Grab a common grid to reproject too and a create a land mask

In [ ]:
gbox = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/5km/WCF_5km_monthly_1982_2022.nc').odc.geobox
gbox

#create a mask of aus extent
mask = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/5km/WCF_5km_monthly_1982_2022.nc')['WCF']
mask = mask.mean('time')
mask = xr.where(mask>-99, 1, 0)

## MODIS NDWI (vegetation water)

The orginal Gao (1996) paper says to use the 1230_1250nm band (band 5 in MODIS), but other sources suggest band 6. On the basis of experiments, looks like B6 is more sensitive.

This takes ~9-10 mins per year to process using 24 cores

In [ ]:
chunks=dict(latitude=1000, longitude=1000, time=1)

#loop through each year
for year in years:
    
    if os.path.exists(results+'NDWI'+'/NDWI_5km_'+year+'.nc'):
            continue
    else:
        print('NDWI', year)
    
    modis_sr_inputs = {
        'SR_B2': 'MODIS/mosaic/MCD43A4.006/MCD43A4.006.b02.500m_0841_0876nm_nbar.'+year+'.nc',
        'SR_B6': 'MODIS/mosaic/MCD43A4.006/MCD43A4.006.b06.500m_1628_1652nm_nbar.'+year+'.nc',
        # 'SR_B5': 'MODIS/mosaic/MCD43A4.006/MCD43A4.006.b05.500m_1230_1250nm_nbar.'+year+'.nc',
        
         }

    d = {}
    for k,i in modis_sr_inputs.items():
        
        #open and do some prelim processing
        ds = xr.open_dataset(base+i, chunks=chunks)
        ds = assign_crs(ds, crs='epsg:4326')
        ds = ds.to_array()
        ds = ds.squeeze().drop_vars('variable')
        ds.attrs['nodata'] = np.nan
        ds = ds.rename(k)        
        d[k] = ds #add to dict
    
    #calculate NDWI (water index)
    ndwi = (d['SR_B2'] - d['SR_B6']) / (d['SR_B2'] + d['SR_B6'])

    #resample time
    ndwi = ndwi.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean().persist()

    # resample spatial
    ndwi = ndwi.odc.reproject(gbox, resampling='average').compute()  # bring into memory
    
    #tidy up
    ndwi = round_coords(ndwi)
    ndwi.attrs['nodata'] = np.nan
    ndwi = ndwi.rename('WI')

    #mask to aus land extent
    ndwi = ndwi.where(mask)
    
    #export result
    folder = '/g/data/os22/chad_tmp/AusEFlux/data/interim/WI'
    if not os.path.exists(folder):
        os.makedirs(folder)

    ndwi.astype('float32').to_netcdf(results+'NDWI'+'/NDWI_5km_'+year+'.nc')


## MODIS LST

Using Aqua land surface temperature (afternoon overpass) i.e. `MYD11A1.006`

This takes ~5 mins per year to process using 24 cores

QC masking MODIS, help: https://spatialthoughts.com/2021/08/19/qa-bands-bitmasks-gee/

decimal to binary converter: https://www.rapidtables.com/convert/number/decimal-to-binary.html


- 0  (decimal): 0 0 0 0 0 0 0 0; LST produced, good quality, good data, emis err < 0.01, LST error <1K
- 5  (decimal): 0 0 0 0 0 1 0 1; LST produced, other quality, other quality, emis err < 0.01, LST error <= 1K
- 17 (decimal): 0 0 0 1 0 0 0 1; LST produced, other quality, good data, emis err < 0.02, LST error <= 1K
- 21 (decimal): 0 0 0 1 0 1 0 1; LST produced, other quality, other quality, emis err < 0.02, LST error <= 1K
- 64 (decimal): 0 1 0 0 0 0 0 0; LST produced, good quality, good data, emis err < 0.01, LST error <= 2K
- 65 (decimal): 0 1 0 0 0 0 0 1; LST produced, other quality, good data, emis err < 0.01, LST error <= 2K
- 81 (decimal): 0 1 0 1 0 0 0 1; LST produced, other quality, other quality, emis err < 0.02, LST error <= 2K
< cores

In [ ]:
%%time
chunks=dict(latitude=500, longitude=500, time=-1)
#loop through each year
for year in years:
    
    modis_sr_inputs = {
        'LST' :'MODIS/mosaic/MYD11A1.006/MYD11A1.006.LST_Day_1km.'+year+'.nc'
         }
    
    for k,i in modis_sr_inputs.items():
         
        if os.path.exists(results+'LST'+'/LST_5km_'+year+'.nc'):
            continue
        else:
            print(k, year)
    
        ds = xr.open_dataset(base+i,chunks=chunks)

        #deal with messed up QC in 2022 (temporary hopefully)
        if year != '2022':
            qc = xr.open_dataset(base+'MODIS/mosaic/MYD11A1.006/MYD11A1.006.QC_Day.'+year+'.nc',
                     chunks=chunks)
            #data is high quality <2k error see above.
            m = xr.where((qc.QC_Day==0) | (qc.QC_Day==5) | (qc.QC_Day==17) | (qc.QC_Day==21) |
                         (qc.QC_Day==64) | (qc.QC_Day==65) | (qc.QC_Day==81), 1, 0)
            
            ds = ds.where(m)

        #tidy up
        ds = assign_crs(ds, crs='epsg:4326')
        ds = ds.to_array()
        ds = ds.squeeze().drop('variable')
        ds.attrs['nodata'] = np.nan

        #resample time and space
        ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean().compute()
        ds = ds.odc.reproject(gbox, resampling='average')

        #tidy up
        ds = round_coords(ds)
        ds.attrs['nodata'] = np.nan
        ds = ds.rename(k)
        ds = ds.where(mask) #land mask

        #convert to celsius
        ds = ds-273.15
        
        #export result
        folder = results+k
        if not os.path.exists(folder):
            os.makedirs(folder)
        ds.to_netcdf(results+k+'/'+k+'_5km_'+year+'.nc')

## OzWALD NDVI & EVI (derived from MODIS)

This takes ~1.5 mins per year to process using 24 cores

NDVI is used derive the vegetation fractions, EVI is used as the predictor in ML model

In [ ]:
%%time
chunks=dict(latitude=1000, longitude=1000, time=-1)

#loop through each year
for year in years:
    
    modis_ndvi = {
        'NDVI' :'OzWALD/8day/NDVI/OzWALD.NDVI.'+year+'.nc',
        'EVI' :'OzWALD/8day/EVI/OzWALD.EVI.'+year+'.nc'
         }
    
    for k,i in modis_ndvi.items():
         
        if os.path.exists(f'{results}{k}/{k}_5km_{year}.nc'):
            continue
        else:
            print(k, year)
        
        ds = xr.open_dataset(base+i,chunks=chunks)
        ds = ds.transpose('time', 'latitude', 'longitude')
        
        #tidy up
        ds = assign_crs(ds, crs='epsg:4326')
        ds = ds.to_array()
        ds = ds.squeeze().drop('variable')
        ds.attrs['nodata'] = np.nan
        
        #resample time and space
        ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean().compute()
        ds = ds.odc.reproject(gbox, resampling='average')
        
        #tidy up
        ds = round_coords(ds)
        ds.attrs['nodata'] = np.nan
        ds = ds.rename(k)
        ds = ds.where(mask) #land mask

        #export result
        folder = results+k
        if not os.path.exists(folder):
            os.makedirs(folder)
        ds.to_netcdf(f'{results}{k}/{k}_5km_{year}.nc')
        

In [ ]:
%%time
chunks=dict(latitude=1000, longitude=1000, time=-1)

#loop through each year
for year in years:
    
    modis_ndvi = {
        'NDVI' :'OzWALD/8day/NDVI/OzWALD.NDVI.'+year+'.nc'
         }
    
    for k,i in modis_ndvi.items():
         
        if os.path.exists(results+'NDVI'+'/NDVI_5km_'+year+'.nc'):
            continue
        else:
            print(k, year)
        
        ds = xr.open_dataset(base+i,chunks=chunks)
        ds = ds.transpose('time', 'latitude', 'longitude')
        
        #tidy up
        ds = assign_crs(ds, crs='epsg:4326')
        ds = ds.to_array()
        ds = ds.squeeze().drop('variable')
        ds.attrs['nodata'] = np.nan
        
        #resample time and space
        ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean().compute()
        ds = ds.odc.reproject(gbox, resampling='average')
        
        #tidy up
        ds = round_coords(ds)
        ds.attrs['nodata'] = np.nan
        ds = ds.rename(k)
        ds = ds.where(mask) #land mask

        #export result
        folder = results+k
        if not os.path.exists(folder):
            os.makedirs(folder)
        ds.to_netcdf(results+k+'/'+k+'_5km_'+year+'.nc')


## Vegetation Height

This was reprojected from 25m to 1 km previously.

In [ ]:
ds = xr.open_dataset('/g/data/os22/chad_tmp/AusEFlux/data/VegH_1km_2007_2010.nc', chunks=dict(x=250, y=250))
ds = assign_crs(ds, crs='epsg:4326')
ds.attrs['nodata'] = np.nan
ds = ds['VegH']
ds = ds.odc.reproject(gbox, resampling='average').compute()
ds = round_coords(ds)

# convert to time-series (same values for each time-step)
# create a dataset for each year just to be consistent with other features
# open another dataset so we can grab the time dim
for year in years:
    
    if os.path.exists(results+'VegH/VegH_5km_'+year+'.nc'):
            continue
    else:
        print(year)
            
    da = xr.open_dataarray(results+'NDWI'+'/NDWI_5km_'+year+'.nc')
    
    #expand time dim using other dataset's time.
    dss = ds.expand_dims(time=da.time)
    
    #mask to aus land extent
    dss = dss.where(mask)
    dss = dss.rename('VegH')
    #export
    dss.to_netcdf(results+'VegH/VegH_5km_'+year+'.nc')

## Climate data


### Rain

This runs very fast, ~6 seconds per year using 24 cores

With rainfall, we need to grab data from a year earlier (from 2002 onwards) because later on we calculate 12-month cumulative rainfall

In [ ]:
chunks=dict(latitude=250, longitude=250, time=-1)

#loop through each year
for year in years:

    clim_inputs = {
        'rain': 'OzWALD/daily/meteo/Pg/OzWALD.daily.Pg.'+year+'.nc'
         }
    
    d = {}
    for k,i in clim_inputs.items():
        
        #open and do some prelim processing
        ds = xr.open_dataset(base+i, chunks=chunks).persist()
        ds = ds.transpose('time', 'latitude', 'longitude')
        ds = assign_crs(ds, crs='epsg:4326')
        ds = ds.to_array()
        ds = ds.squeeze().drop_vars('variable')
        ds.attrs['nodata'] = np.nan
        ds = ds.rename(k)        
        d[k] = ds #add to dict
            
    if os.path.exists(f'{results}/{k}/{k}_5km_{year}.nc'):
        continue
    else:
        print(k, year)
            
    #resample time, bring into memory
    ds = d['rain'].resample(time='MS', loffset=pd.Timedelta(14, 'd')).sum().compute()

    # resample spatial
    ds = ds.odc.reproject(gbox, resampling='nearest')
    
    #tidy up
    ds = round_coords(ds)
    ds.attrs['nodata'] = np.nan
    ds = ds.rename(k)

    #mask to aus land extent
    ds = ds.where(mask)

    #export result
    folder = results+k
    if not os.path.exists(folder):
        os.makedirs(folder)

    ds.astype('float32').to_netcdf(f'{results}/{k}/{k}_5km_{year}.nc')


### Tavg

The temperature datasets are daily and 500m resolution, Tavg can only be calculated as `Tavg = Tmin + kTavg*(Tmax - Tmin)` and each variable is stored as one large netcdf per year on file (this limits parallelization in loading the datasets) so the memory and compute requirements are v. large.  

This process is so slow we need to break into up into two steps.
1. Iteratively load each of the three variables we need to calculate Tavg, reproject them to 5km resolution and save to disk
2. Calculate Tavg and save to disk

Step 1 takes ~ 87 mins per year to run using 24 cores.

Step 2. takes ~ 10-20 seconds per year

#### Step 1

In [ ]:
%%time
chunks=dict(latitude=10000, longitude=10000, time=1) #ie one chunk per time

#loop through each year
for year in years:

    clim_inputs = {
        'Tmin':'OzWALD/daily/meteo/Tmin/OzWALD.Tmin.'+year+'.nc', 
        'Tmax':'OzWALD/daily/meteo/Tmax/OzWALD.Tmax.'+year+'.nc',
        'kTavg':'OzWALD/daily/meteo/kTavg/OzWALD.kTavg.'+year+'.nc'
         }
    
    for k,i in clim_inputs.items():
        
        if os.path.exists(f'{results}/{k}/{k}_5km_{year}.nc'):
            continue
        else:
            print(k, year)
        
        #open and do some prelim processing
        ds = xr.open_dataset(base+i, chunks=chunks) # open as one chunk per time
        ds = assign_crs(ds, crs='epsg:4326')
        ds = ds.to_array()
        ds = ds.squeeze().drop_vars('variable')
        ds.attrs['nodata'] = np.nan
        #ds = ds.chunk(latitude=10000, longitude=10000, time=1) # now rechunk for the reproject
        
        #we need to spatial resample first to reduce RAM/speed up.
        if k=='kTavg':
            #upscaling from 10km to 5km
            ds = ds.odc.reproject(gbox, resampling='nearest').compute()
            ds = round_coords(ds)
        else:
            # downsacling from 500m to 5km
            ds = ds.odc.reproject(gbox, resampling='average').compute()
            ds = round_coords(ds)

        #tidy up
        ds = ds.transpose('time', 'latitude', 'longitude')
        ds = ds.rename(k)
        ds = assign_crs(ds, crs='epsg:4326')
        
        # #export result
        folder = results+k
        if not os.path.exists(folder):
            os.makedirs(folder)
        
        ds.astype('float32').to_netcdf(f'{results}/{k}/{k}_5km_{year}.nc')
        

#### Step 2

no need for dask now

In [ ]:
%%time
#loop through each year
for year in years:

    clim_inputs = {
        'Tmin':f'{results}/Tmin/Tmin_5km_{year}.nc', 
        'Tmax':f'{results}/Tmax/Tmax_5km_{year}.nc',
        'kTavg':f'{results}/kTavg/kTavg_5km_{year}.nc'
         }
    
    if os.path.exists(f'{results}/Tavg/Tavg_5km_{year}.nc'):
            continue
    else:
        print('Tavg', year)
    
    d={}
    for k,i in clim_inputs.items():
        ds = xr.open_dataarray(i)
        d[k] = ds
    
    #calculate tavg
    ds = d['Tmin'] + d['kTavg']*(d['Tmax'] - d['Tmin'])

    #resample time
    ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean()
    
    #tidy up
    ds.attrs['nodata'] = np.nan
    ds = ds.rename('Tavg')

    #mask to aus land extent
    ds = ds.where(mask)

    #export result
    folder = '/g/data/os22/chad_tmp/AusEFlux/data/interim/Tavg/'
    if not os.path.exists(folder):
        os.makedirs(folder)

    ds.astype('float32').to_netcdf(f'{results}/Tavg/Tavg_5km_{year}.nc')

### Solar radiation

Incoming shortwave rad (MJ/m2/d), using SILO version https://dapds00.nci.org.au/thredds/catalog/ub8/au/SILO/radiation/catalog.html

Can account for the effect of topography on incoming radiation in the same way as OzWALD, where it is done by multiplying incoming radiation with the grids:`
//g/data/xc0/project/OzWALD/R2021/model/static/SWratio_500m_*.`nc grids. Where the wild card is the month of the year from 01:12 - This is fast, 5 seconds per year



In [ ]:
%%time
chunks=dict(lat=250, lon=250, time=-1)

#loop through each year
for year in years:
    
        clim_inputs = {
            'SRAD':'SILO/radiation/'+year+'.radiation.nc'
             }
        
        for k,i in clim_inputs.items():
            
            if os.path.exists(f'{results}/{k}/{k}_5km_{year}.nc'):
                continue
            else:
                print(k, year)
            
            #open and do some prelim processing
            ds = xr.open_dataset(base+i).drop('crs').chunk(chunks)
            ds = assign_crs(ds, crs='epsg:4326')
            ds = ds.to_array()
            ds = ds.squeeze().drop_vars('variable')
            ds.attrs['nodata'] = np.nan
    
            # resample time and space
            ds = ds.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean()
            ds = ds.odc.reproject(gbox, resampling='nearest').compute()
            ds = round_coords(ds)
            
            # tidy up and mask land
            ds = ds.rename(k)
            ds = assign_crs(ds, crs='epsg:4326')
            ds = ds.where(mask)
    
            # # #export result
            folder = results+k
            if not os.path.exists(folder):
                os.makedirs(folder)
            
            ds.astype('float32').to_netcdf(f'{results}/{k}/{k}_5km_{year}.nc')


### Vapour Pressure Deficit

Using SILO version for VP: https://dapds00.nci.org.au/thredds/catalog/ub8/au/SILO/vp/catalog.html

Calculating VPD requires air temperature, so this must be run after Tavg has been computed.
 - This is fast, 5 seconds per year



In [ ]:
%%time
chunks=dict(lat=250, lon=250, time=-1)

#loop through each year
for year in years:
    
        clim_inputs = {
            'VPD':'SILO/vp/'+year+'.vp.nc'
             }
        
        for k,i in clim_inputs.items():
            
            if os.path.exists(f'{results}/{k}/{k}_5km_{year}.nc'):
                continue
            else:
                print(k, year)
            
            #open and do some prelim processing
            vp = xr.open_dataset(base+i).drop('crs').chunk(chunks)
            vp = assign_crs(vp, crs='epsg:4326')
            vp = vp.to_array()
            vp = vp.squeeze().drop_vars('variable')
            vp.attrs['nodata'] = np.nan
    
            # resample time and space
            vp = vp.resample(time='MS', loffset=pd.Timedelta(14, 'd')).mean()
            vp = vp.odc.reproject(gbox, resampling='nearest').compute()
            vp = round_coords(vp)
            
            # mask land
            vp = assign_crs(vp, crs='epsg:4326')
            vp = vp.where(mask)

            #calculate VPD
            ta = xr.open_dataarray(f'{results}/Tavg/Tavg_5km_{year}.nc')
            sat_vp = (6.11 * np.exp((2500000/461) * (1/273 - 1/(273 + ta))))
            ds = sat_vp - vp

            # tidy up
            ds = ds.rename(k)
            ds.attrs['nodata'] = np.nan
            ds.attrs['units'] = 'hPa'
    
            # export result
            folder = results+k
            if not os.path.exists(folder):
                os.makedirs(folder)
            
            ds.astype('float32').to_netcdf(f'{results}/{k}/{k}_5km_{year}.nc')
